In [1]:
#Data Structure Lib
import numpy as np
from collections import OrderedDict
#Data Parser Lib
import os
import glob
#Data Zipped Lib
import gzip
import joblib
import pickle
"""Machine Learning Library"""
#Train and Test data split
from sklearn.model_selection import train_test_split
from ga_selection import GASelection
from SVMranking import SVMranking
from SVM_classfy import SVM_classfy
from arranged_case import arranged_case
from feature_normalize import preprocessing_date
#PCA
from generic_pca import gpca
#COPY
import copy
## Config File
import config
# Classfiication Report
from sklearn.metrics import classification_report
#Plot Lib
from plot_result import plot_result
#output json parser
from result_json_parser import AutoML_Config as ac
from datetime import datetime
####################

In [2]:
with gzip.open('pre_features_n_zip.pkl', 'rb') as f:
    features_n =  pickle.load(f)
with gzip.open('pre_classes_zip.pkl', 'rb') as f:
    classes =  pickle.load(f)
with gzip.open('feature_names_zip.pkl', 'rb') as f:
    feature_names =  pickle.load(f)

In [3]:
print(feature_names)

['Peak value', 'RMS', 'Kurtosis', 'Crest factor', 'Clearance factor', 'Impulse factor', 'Shape factor', 'Skewness', 'SMR', '5th normalized moment', '6th nomalized moment', 'Mean', 'Shape factor2', 'Peak to peak', 'Kurtosis factor', 'Standard deviation', 'Smoothness', 'Uniformity', 'Normal negative log-likelihood', 'Entropy estimation value', 'Entropy estimation error value', 'Histogram upper bound', 'Histogram lower bound', 'Frequency center', 'Mean Square Frequency', 'Rms of frequency', 'Variance frequency', 'Root variance frequency', 'Spectrum overall', 'Spectrum rms overall']


In [4]:
conf = config.loadConfig('config.json')

In [5]:
hist, selfeat, shist = GASelection(feature_n=features_n, 
                                                            classes=classes, 
                                                            cv=conf['GA']['cv'],
                                                            scoring=conf['GA']['scoring'],
                                                            max_features=conf['GA']['max_features'],
                                                            n_population=conf['GA']['n_population'],
                                                            crossover_proba=conf['GA']['crossover_proba'],
                                                            mutation_proba=conf['GA']['mutation_proba'],
                                                            n_generations=conf['GA']['n_generations'],
                                                            crossover_independent_proba=conf['GA']['crossover_independent_proba'],
                                                            mutation_independent_proba=conf['GA']['mutation_independent_proba'],
                                                            tournament_size=conf['GA']['tournament_size'],
                                                            n_gen_no_change=conf['GA']['n_gen_no_change'],
                                                            caching=conf['GA']['caching'])

In [6]:
feature_names = np.array(feature_names)
sel_feature_names = feature_names[selfeat] # 선택된 특징 이름들 array
print(f"Select Feature: {sel_feature_names}")

Select Feature: ['Crest factor' 'Entropy estimation value' 'Histogram upper bound']


In [7]:
selfeat = selfeat.flatten()
selfeat

array([False, False, False,  True, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False,  True, False,  True, False, False, False, False, False,
       False, False, False])

In [62]:
print("\nSupport Vector Machine Hyperparametor as followed")
for k, v in conf['SVM'].items():
    print(f"{k}: {v}")
selfeature = features_n[:, selfeat]
print(selfeature.shape)


Support Vector Machine Hyperparametor as followed
C: [0.1, 0.1]
gamma: [0.1, 0.1]
n_splits: 1
n_iter: 100
test_size: 0.33
search: random
(14784, 3)


In [ ]:

best_accuracy, predict_result, confmat, estimator = SVM_classfy(
                                                                                features=selfeature, 
                                                                                classes=classes.ravel(), 
                                                                                **conf['SVM']
                                                                                )
svm_classfy= {}
svm_classfy['best_accuracy'] = best_accuracy
svm_classfy['predict_result'] = predict_result
svm_classfy['confmat'] = confmat
svm_classfy['estimator'] = estimator
with gzip.open('svm_classfy_zip.pkl', 'wb') as f:
    pickle.dump(svm_classfy, f)

In [13]:
with gzip.open('svm_classfy_zip.pkl', 'rb') as f:
    svm_classfy =  pickle.load(f)
best_accuracy = svm_classfy['best_accuracy']
predict_result = svm_classfy['predict_result']
confmat = svm_classfy['confmat']
estimator = svm_classfy['estimator']

In [19]:
estimator.best_estimator_.n_support_

array([ 950, 3254, 2163, 3143])

In [20]:
a = classification_report(
                            estimator.best_estimator_.predict(selfeature), 
                            classes.ravel(), 
                            target_names=list(conf['input']['target_List'].keys()),
                            output_dict=True)
print(classification_report(
                            estimator.best_estimator_.predict(selfeature), 
                            classes.ravel(), 
                            target_names=list(conf['input']['target_List'].keys())))

c:\miniconda3\envs\py37mattopy\lib\site-packages\sklearn\metrics\_classification.py:1268: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

      Normal       1.00      0.26      0.41     14784
     Rubbing       0.00      0.00      0.00         0
   Unbalance       0.00      0.00      0.00         0
Misalignment       0.00      0.00      0.00         0

    accuracy                           0.26     14784
   macro avg       0.25      0.06      0.10     14784
weighted avg       1.00      0.26      0.41     14784



c:\miniconda3\envs\py37mattopy\lib\site-packages\sklearn\metrics\_classification.py:1268: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
a

In [21]:
Rank, combination = SVMranking(confmat)
print(combination)
print(Rank)


[[   1    3   62 1075]
 [   1    2  210 2541]
 [   1    4  466 1268]
 [   3    4  694 1825]
 [   2    3 1296 3098]
 [   2    4 1638 3291]]
[[   2    4 1638 3291]
 [   2    3 1296 3098]
 [   3    4  694 1825]
 [   1    4  466 1268]
 [   1    2  210 2541]
 [   1    3   62 1075]]


In [75]:
print(features_n.shape)
print(classes.shape)
print(Rank.shape)
feature_re, class_re, c = arranged_case(features_n, classes, Rank, 1)

(14784, 30)
(14784,)
(6, 4)


IndexError: too many indices for array

In [76]:
a, b = gpca(selfeature)#selfeature

In [84]:
from sklearn.decomposition import PCA
pca = PCA(n_components=3, svd_solver='full')
Trainfeature = pca.fit_transform(features_n)

In [85]:
Trainfeature[:3]

array([[-5.63230429e+12,  6.09188854e+05,  3.64247996e+02],
       [-6.10218654e+12,  6.22031023e+05, -8.32984181e+02],
       [-5.29090389e+12,  5.91323176e+05,  4.53259367e+03]])

In [86]:
a[:3]

array([[ 4.39604163e+00, -5.19021676e-02,  7.34619001e-04],
       [ 3.27193203e+00,  1.33326369e-01,  2.51928498e-03],
       [ 4.41142625e+00, -9.88991449e-02,  1.54327381e-03]])

In [71]:
Trainfeature.shape

(14784, 3)

In [74]:
np.hstack((selfeature, Trainfeature)).shape


(14784, 6)

In [66]:
np.sort(Trainfeature, axis=0)

array([[-6.83922760e+12, -3.92521766e+06, -2.46579537e+05],
       [-6.83904528e+12, -3.90545170e+06, -2.42531491e+05],
       [-6.83899884e+12, -3.90425305e+06, -1.93021899e+05],
       ...,
       [ 3.21673292e+14,  7.18857008e+06,  2.19666984e+05],
       [ 3.44697481e+14,  8.22501948e+06,  2.26786458e+05],
       [ 3.58369750e+14,  8.62684388e+06,  2.65387413e+05]])